# Load data

Leyendo el csv

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/flipkart_review_data_2022_02.csv")
df = df.drop(columns="Unnamed: 0")

In [3]:
# Total de elementos
df.shape

(343, 14)

In [4]:
df.head(3)

,ProductUrl,productTitle,productPrice,averageRating,reviewTitle,reviewDescription,reviewAuthor,reviewAt,reviewLikes,reviewDislikes,certifiedBuyer,reviewerLocation,scrapedAt,uniqId
0,https://www.flipkart.com/lg-108-cm-43-inch-ult...,LG 108 cm (43 inch) Ultra HD (4K) LED Smart TV,"₹36,499",4.4,Excellent,Very good product,palakollu komali,6 months ago,0,1,True,Tenali,20/02/2022 02:03:42,4e9df8d3-736c-5dae-b867-b92b7f09fc84
1,https://www.flipkart.com/lg-108-cm-43-inch-ult...,LG 108 cm (43 inch) Ultra HD (4K) LED Smart TV,"₹36,499",4.4,Really Nice,Nice and super picture quality.and sound also ...,Yogesh Virkar,3 months ago,0,0,True,Mumbai,20/02/2022 02:03:42,7ba04258-9f05-5642-ba33-8982db1f6c5e
2,https://www.flipkart.com/lg-108-cm-43-inch-ult...,LG 108 cm (43 inch) Ultra HD (4K) LED Smart TV,"₹36,499",4.4,Super!,This is excellent picture qualityUltimate soun...,Sudhir Kumar Chaudhary,3 months ago,0,0,True,Haridwar,20/02/2022 02:03:42,bfb20e97-eacf-5b76-a5eb-91999d6bd7a3


A primera vista, la variable objetivo podría ser la columna "averageRating" o "reviewTitle"

In [5]:
# Conteo inicial de la variable 'target'
df.groupby("averageRating")["averageRating"].count()

averageRating
4.4    343
Name: averageRating, dtype: int64

In [6]:
# Revisando los datos, es necesario organizarlo en positive/negative
# y disminuir la cardinalidad de la tabla
df.groupby("reviewTitle")["reviewTitle"].count().sort_values(ascending=False).head(10)

reviewTitle
Classy product         22
Great product          20
Wonderful              17
Perfect product!       15
Best in the market!    14
Highly recommended     14
Must buy!              13
Excellent              13
Worth the money        13
Worth every penny      12
Name: reviewTitle, dtype: int64

In [7]:
# Podríamos filtrar los comentarios por likes
# dejando los más relevantes, o eliminado aquellos
# con mayor dislikes
df.describe()

,averageRating,reviewLikes,reviewDislikes
count,3.430000e+02,343.000000,343.000000
mean,4.400000e+00,4.309038,1.755102
std,1.778952e-15,15.885670,6.466610
min,4.400000e+00,0.000000,0.000000
25%,4.400000e+00,0.000000,0.000000
50%,4.400000e+00,0.000000,0.000000
75%,4.400000e+00,3.000000,1.000000
max,4.400000e+00,235.000000,64.000000


In [8]:
# Eliminaré el 5% de comentarios con más dislikes
q_05_dislikes = df["reviewDislikes"].quantile(0.99)
q_05_dislikes

35.0

In [9]:
df = df[df.reviewDislikes < q_05_dislikes]

In [10]:
# Después del filtrado nos quedamos con 325 comentarios
df.shape

(338, 14)

In [11]:
# Creando dos etiquetas: Bueno y Malo

# [1] Obteniendo valores únicos

unique_titles = df["reviewTitle"].unique()
unique_titles

array(['Excellent', 'Really Nice', 'Super!', 'Just wow!',
       'Highly recommended', 'Great product', 'Worth the money',
       'Good quality product', 'Nice', 'Decent product',
       'Perfect product!', 'Classy product', 'Terrific', 'Good choice',
       'Fabulous!', 'Worth every penny', 'Wonderful', 'Simply awesome',
       'Very Good', 'Terrific purchase', 'Brilliant',
       'Mind-blowing purchase', 'Must buy!', 'Pretty good', 'Awesome',
       'Best in the market!', 'Value-for-money', 'Fair', 'Hated it!',
       'Just okay', 'Could be way better', 'Useless product',
       'Unsatisfactory', 'Not good', 'Delightful', 'Good', 'Nice product',
       'Very poor', 'Moderate', 'Does the job', 'Utterly Disappointed',
       'Worst experience ever!', 'Expected a better product',
       'Not recommended at all', 'Waste of money!', 'Absolute rubbish!',
       'Did not meet expectations'], dtype=object)

In [12]:
# Etiquetado manual de la información
new_categories_dict = {
    "good": ['Excellent', 'Really Nice', 'Super!', 'Just wow!',
       'Highly recommended', 'Great product', 'Worth the money',
       'Good quality product', 'Nice', 'Decent product',
       'Perfect product!', 'Classy product', 'Terrific', 'Good choice',
       'Fabulous!', 'Worth every penny', 'Wonderful', 'Simply awesome',
       'Very Good', 'Terrific purchase', 'Brilliant',
       'Mind-blowing purchase', 'Must buy!', 'Pretty good', 'Awesome',
       'Best in the market!', 'Value-for-money', 'Fair', 'Nice product',
       'Delightful', 'Good', 'Does the job'],

    "bad": ['Slightly disappointed', 'Hated it!', 'Just okay',
            'Could be way better', 'Useless product', 'Unsatisfactory',
            'Very poor', 'Moderate', 'Not good',
            'Utterly Disappointed', 'Worst experience ever!',
            'Expected a better product', 'Not recommended at all',
            'Waste of money!', 'Absolute rubbish!',
            'Did not meet expectations', "Hated it!",
            "Not good", "Useless product"]
}

def relabel_target(title: pd.Series):
  """Returns a new category according to the new labes dictionary"""
  new_category = "not found"

  for category in new_categories_dict.keys():
    if title in new_categories_dict[category]:
      new_category = category
      break

  return new_category

In [13]:
df["sentiment"] = df["reviewTitle"].apply(relabel_target)

In [14]:
# Nueva distribución de categorias
# Lo primero que podemos observar es que se encuentra
# Imbalanceado
df.groupby("sentiment")["sentiment"].count()

sentiment
bad      20
good    318
Name: sentiment, dtype: int64

# Train, Test

In [15]:
from sklearn.model_selection import train_test_split

# Normalmente se usa un 80-20 o 70-30 para
# para el modelo, pero ya que son muy pocos datos,
# se utilizará un muestra pequena para testear los datos
train, test = train_test_split(df, test_size=0.2,
                               random_state=0, stratify=df["sentiment"])

In [16]:
train.groupby("sentiment")["sentiment"].count()

sentiment
bad      16
good    254
Name: sentiment, dtype: int64

In [17]:
round(train.groupby("sentiment")["sentiment"].count() / train.shape[0] * 100, 2)

sentiment
bad      5.93
good    94.07
Name: sentiment, dtype: float64

In [18]:
test.groupby("sentiment")["sentiment"].count()

sentiment
bad      4
good    64
Name: sentiment, dtype: int64

In [19]:
round(test.groupby("sentiment")["sentiment"].count() / test.shape[0] * 100, 2)

sentiment
bad      5.88
good    94.12
Name: sentiment, dtype: float64

Para tratar datos no balanceados, podemos:
- Realizar un sub-muestreo de la data que tiene mayor cantidad de datos
- Realizar un sobre-muestreo de la etiqueta con menor cantidad de datos
- Usar K-fold cross validation
- Ensamblado de diferentes modelos, realizando un submuestreo de la categoría con mayor cantidad de datos
y siempre usando la categoría con menos datos

Por el tiempo disponible optare por K-fold cross validation que se integra dentro GridSearch.

# Preprocess (data cleaning)

La columna reviewDescription será utilizada como feature para la construcción del modelo, el preprocesamiento incluye:

- Eliminar signos de puntuación
- Convertir a minúsculas
- Remover palabras más comunes (stopwords)
- Lematización: llevar a la raiz
- Remover palabras menores a 2 letras
- Vectorizar la información


In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [21]:
# Descargando librería stopwords
nltk.download('stopwords')
# Descargando librería para lematización
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
# Obteniendo stopwords para idioma inglés
STOPWORDS = nltk.corpus.stopwords.words('english')

In [23]:
# Instalando librería para eliminar emojis
!pip install emoji

In [24]:
import emoji

In [25]:
LEMMATIZER = WordNetLemmatizer()

def remove_stopwords(text: str) -> str:
  text_processed = " ".join(word for word in text.split(" ") if word not in STOPWORDS)
  return text_processed


def remove_punctuation(text: str) -> str:
  remove_chars_list = ["?", ",", ".", ";", ":",  "!",'"']

  for char in remove_chars_list:
    text = text.replace(char, " ")

  return text


def lemmatisation(text: str) -> str:
  return " ".join(LEMMATIZER.lemmatize(word) for word in text.split(" "))


def remove_emoji(text: str) -> str:
  return emoji.replace_emoji(text, '')


def remove_short_words(text: str) -> str:
  return " ".join(word for word in text.split(" ") if len(word) > 2)


def encode_categories(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df["sentiment"] == "good", "target"] = 1
  df.loc[df["sentiment"] == "bad", "target"] = 0

  return df

In [26]:
def process_text(df: pd.DataFrame) -> pd.DataFrame:
  # [0] Combinando titulo y la descripción
  df["feature"] = df["reviewTitle"]  + " " + df["reviewDescription"]
  # [1] Eliminando signos de puntuación
  df["feature"] = df['feature'].apply(remove_punctuation)
  # [2] Convirtiendo a minúscula
  df["feature"] = df['feature'].str.lower()
  # [3] Eliminando las palabras más comunes
  df["feature"] = df['feature'].apply(remove_stopwords)
  # [4] Lematización
  df["feature"] = df['feature'].apply(lemmatisation)
  # [5] Removiendo emojis
  df["feature"] = df['feature'].apply(remove_emoji)
  # [6] Filtrando palabras muy cortas
  df["feature"] = df['feature'].apply(remove_short_words)
  # [7] Eliminando espacios en blanco al inicio y final del texto
  df["feature"] = df['feature'].str.strip()

  return df

In [27]:
# Aplicando el procesamiento a datos de entrenamiento
train_processed = process_text(train)
train_processed = encode_categories(train_processed)

In [28]:
# Revisión de la limpieza del texto
train_processed[["reviewTitle", "reviewDescription", "feature", "target"]].sample(5)

,reviewTitle,reviewDescription,feature,target
220,Terrific purchase,"Super TV #Lg , thanks lg, thank u flipkart Do...",terrific purchase super #lg thanks thank flipk...,1.0
209,Fabulous!,Good,fabulous good,1.0
125,Great product,Very Nice Video Quality. Good Sound. Good User...,great product nice video quality good sound go...,1.0
149,Value-for-money,Good deal on flipkart.,value-for-money good deal flipkart,1.0
44,Brilliant,Product and service excellent,brilliant product service excellent,1.0


# Information Processing (vectorization)

A continuación procedere a vectorizar el texto para que pueda alimentar al modelo.

In [29]:
# Term Frequency Inverse Document Frequency
# Considera el peso de las palabras y es una opción
# popular para la vectorización del text
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorizer = TfidfVectorizer()
train_vec = vectorizer.fit_transform(train_processed["feature"])

In [31]:
df_x_train = pd.DataFrame.sparse.from_spmatrix(train_vec)
df_x_train.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
244,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
231,0.0,0.0,0.0,0.146953,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.129715
191,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
50,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
243,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [32]:
# Guardar el vectorizar en un archivo pickle
import pickle

In [33]:
VECTORIZER_PATH = '/content/vectorizer'
file = open(VECTORIZER_PATH, 'wb')
pickle.dump(vectorizer, file)
file.close()

In [34]:
from typing import Any

def save_to_pickle(obj: Any, path: str) -> None:
  file = open(path, 'wb')
  pickle.dump(obj, file)
  file.close()

In [35]:
def vectorize_features(df: pd.DataFrame, ) -> pd.DataFrame:
  file = open(VECTORIZER_PATH, 'rb')
  vectorizer = pickle.load(file)
  file.close()

  train_vec = vectorizer.transform(df)
  df_vectorized = pd.DataFrame.sparse.from_spmatrix(train_vec)

  return df_vectorized

In [36]:
# Resumen del vocabulario aprendido
# data_vec = {k: [v] for k, v in vectorizer.vocabulary_.items()}
# pd.DataFrame.from_dict(data_vec, orient="index").sort_values(0, ascending=False).head(10)

In [37]:
# [1] Preprocseamiento (limpieza del texto)
test_processed = process_text(test)
# [2] Vectorización
df_x_test = vectorize_features(test_processed["feature"])

In [38]:
df_x_test.head()

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.258217,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


# Model training

## Logistic Regression


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

In [40]:
# Obteniendo y_train para el entrenamiento del modelo
y_train = train_processed["target"]

In [59]:
parameters = {
    'penalty' : ['l1','l2'],
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

logreg = LogisticRegression()

# GridSearchCV
# Crea un modelo por cada combinación del conjunto de parametros
# definidos, y los evalúa usando cross validation.
# Genera y evalua un total de 210 modelos y al final elegimos el mejor

logreg_gs = GridSearchCV(
    estimator=logreg,
    param_grid = parameters,
    # Una buen métrica para evaluar el modelo es f1-score,
    # ya que los datos se encuentran desbalaceados
    scoring='f1_macro',
    cv=5,
    n_jobs=-3,
    verbose=10,
)

logreg_gs.fit(df_x_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV 1/5; 1/42] START C=0.001, penalty=l1, solver=newton-cg......................
[CV 1/5; 1/42] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.4s
[CV 2/5; 1/42] START C=0.001, penalty=l1, solver=newton-cg......................
[CV 2/5; 1/42] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.4s
[CV 3/5; 1/42] START C=0.001, penalty=l1, solver=newton-cg......................
[CV 3/5; 1/42] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.6s
[CV 4/5; 1/42] START C=0.001, penalty=l1, solver=newton-cg......................
[CV 4/5; 1/42] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.3s
[CV 5/5; 1/42] START C=0.001, penalty=l1, solver=newton-cg......................
[CV 5/5; 1/42] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.2s
[CV 1/5; 2/42] START C=0.001, penalty=l1, solver=lbfgs..........................
[CV 1/5; 2/4

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-3,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='f1_macro', verbose=10)

In [60]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

In [61]:
# Aplicando a los datos de test
y_pred_train = logreg_gs.best_estimator_.predict(df_x_train)

# En la matriz de confusión vemos que
# logra predecir correctamente los 16 comentarios negativos
confusion_matrix(y_pred_train, y_train)

array([[ 16,   0],
       [  0, 254]])

In [62]:
print(classification_report(y_pred_train, y_train))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        16
         1.0       1.00      1.00      1.00       254

    accuracy                           1.00       270
   macro avg       1.00      1.00      1.00       270
weighted avg       1.00      1.00      1.00       270



In [63]:
y_pred_train = logreg_gs.best_estimator_.predict(df_x_train)

In [64]:
train_score = f1_score(y_train, y_pred_train, average="macro")
print("Tuned Hyperparameters :", logreg_gs.best_params_)
# Los valores de f1-score van de 0 a 1, mientras
# más cercano sea a 1, mejor será el modelo
print("F1 score en Train:", round(train_score, 2))

Tuned Hyperparameters : {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
F1 score en Train: 1.0


Tiene un alto score, por lo que es posible que se encuentre sobre entrenado.

### Validation

In [65]:
# [1] Preprocesamiento (limpieza del texto)
test_processed = process_text(test)
# [2] Vectorización
x_test = test_processed["feature"]
df_x_test = vectorize_features(x_test)
# Aplicando a los datos de test
y_pred_test = logreg_gs.best_estimator_.predict(df_x_test)

In [66]:
test_processed = encode_categories(test)
y_test = test_processed["target"]

In [67]:
# En la matriz de confusión vemos que solo
# logra predecir 1 de los 4 comentarios negativos.
confusion_matrix(y_pred_test, y_test)

array([[ 1,  0],
       [ 3, 64]])

In [69]:
print(classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

         0.0       0.25      1.00      0.40         1
         1.0       1.00      0.96      0.98        67

    accuracy                           0.96        68
   macro avg       0.62      0.98      0.69        68
weighted avg       0.99      0.96      0.97        68



In [70]:
test_score = f1_score(y_test, y_pred_test, average="macro")

In [71]:
print(f"Train score: {round(train_score, 2)} \nTest score: {round(test_score, 2)}")

Train score: 1.0 
Test score: 0.69


El score de entrenamiento es muy alto, mientras que en los datos de test es de 0.7, es decir, más bajo, por lo que podemos determinar que hay overfitting.

Este problema podrá tratarse en otra iteración, con el uso de más datos,aumentado el número de k-folds, o utilizando un ensamblado de modelos.

Por ahora, el modelo tiene un desempeño acetable, ya que tiene un **score en los datos de Test de 69%**.

In [72]:
# Guardando el modelo
LR_ESTIMATOR_PATH = "/content/lr_estimator"
save_to_pickle(logreg_gs.best_estimator_, LR_ESTIMATOR_PATH)

## XGBoost

In [74]:
# import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [75]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': range(2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

# En total se prueban 480 modelos
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=10,
)

grid_search.fit(df_x_train, y_train)

#grid_search.best_estimator_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(60, 220, 40)},
             scoring='f1_macro', verbose=10)

In [76]:
y_pred_train = grid_search.best_estimator_.predict(df_x_train)

train_score = f1_score(y_train, y_pred_train, average="macro")

In [77]:
print("Tuned Hyperparameters :", grid_search.best_params_)
print(f"F1-score Train: {train_score}")

Tuned Hyperparameters : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 60}
F1-score Train: 0.9025341130604289


### Validación

In [78]:
y_pred = grid_search.best_estimator_.predict(df_x_test)

In [79]:
confusion_matrix(y_pred, y_test)

array([[ 0,  0],
       [ 4, 64]])

In [80]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.94      0.97        68

    accuracy                           0.94        68
   macro avg       0.50      0.47      0.48        68
weighted avg       1.00      0.94      0.97        68



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
test_score = f1_score(y_test, y_pred, average="macro")

In [82]:
print(f"Train score: {train_score} \nTest score: {test_score}")

Train score: 0.9025341130604289 
Test score: 0.48484848484848486


Al revisar los scores, es posible decir que también se dio un overfitting, o sobreentreamiento de los modelos. Se ajusto muy bien en los datos de train, pero no tuvo un buen desempeño en los datos de test.

In [83]:
# Guardando el modelo
XGB_ESTIMATOR_PATH = "/content/xgb_estimator"
save_to_pickle(logreg_gs.best_estimator_, LR_ESTIMATOR_PATH)

Comparando el score para ambos modelos (Logistic Regression y XGB), el que tuvo mejor desempeño fue **Logistic Regession**, debido a tuvo un mejor score en los datos de test.

# Function to process and predict new data

In [86]:
#!/usr/bin/python3
#-*- coding: utf-8 -*-

"""Programa que realiza determinar si una reseña es positiv o negativa para"""

__author__ = "Cuauhtémoc"
__version__ = "1.0"
__status__ = "Development"

import pandas as pd

import emoji
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

# Descargando librería stopwords
nltk.download('stopwords')
# Descargando librería para lematización
nltk.download('wordnet')

# Obteniendo stopwords para idioma inglés
STOPWORDS = nltk.corpus.stopwords.words('english')
VECTORIZER_PATH = '/content/vectorizer'
LR_ESTIMATOR_PATH = "/content/lr_estimator"
XGB_ESTIMATOR_PATH = "/content/xgb_estimator"
BEST_MODEL_PATH = LR_ESTIMATOR_PATH


LEMMATIZER = WordNetLemmatizer()

def remove_stopwords(text: str) -> str:
  text_processed = " ".join(word for word in text.split(" ") if word not in STOPWORDS)
  return text_processed


def remove_punctuation(text: str) -> str:
  remove_chars_list = ["?", ",", ".", ";", ":",  "!",'"']

  for char in remove_chars_list:
    text = text.replace(char, " ")

  return text


def lemmatisation(text: str) -> str:
  return " ".join(LEMMATIZER.lemmatize(word) for word in text.split(" "))


def remove_emoji(text: str) -> str:
  return emoji.replace_emoji(text, '')


def remove_short_words(text: str) -> str:
  return " ".join(word for word in text.split(" ") if len(word) > 2)


def encode_categories(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df["sentiment"] == "good", "target"] = 1
  df.loc[df["sentiment"] == "bad", "target"] = 0

  return df


def process_text(df: pd.DataFrame) -> pd.DataFrame:
  # [0] Combinando titulo y la descripción
  df["feature"] = df["reviewTitle"]  + " " + df["reviewDescription"]
  # [1] Eliminando signos de puntuación
  df["feature"] = df['feature'].apply(remove_punctuation)
  # [2] Convirtiendo a minúscula
  df["feature"] = df['feature'].str.lower()
  # [3] Eliminando las palabras más comunes
  df["feature"] = df['feature'].apply(remove_stopwords)
  # [4] Lematización
  df["feature"] = df['feature'].apply(lemmatisation)
  # [5] Removiendo emojis
  df["feature"] = df['feature'].apply(remove_emoji)
  # [6] Filtrando palabras muy cortas
  df["feature"] = df['feature'].apply(remove_short_words)
  # [7] Eliminando espacios en blanco al inicio y final del texto
  df["feature"] = df['feature'].str.strip()

  return df


def vectorize_features(df: pd.DataFrame, ) -> pd.DataFrame:
  file = open(VECTORIZER_PATH, 'rb')
  vectorizer = pickle.load(file)
  file.close()

  train_vec = vectorizer.transform(df)
  df_vectorized = pd.DataFrame.sparse.from_spmatrix(train_vec)

  return df_vectorized

# Flujo para cargar el modelo y realizar predicción
import sklearn
def load_model():
  file = open(BEST_MODEL_PATH, 'rb')
  estimator = pickle.load(file)
  file.close()

  return estimator


def main(df):

  # [1] Preprocseamiento (limpieza del texto)
  df_processed = process_text(df)
  # [2] Vectorización
  features = vectorize_features(df_processed["feature"])
  # [3] Cargando el modelo
  estimator = load_model()
  # [4] Realizando la predicción
  y = estimator.predict(features)[0]

  sentiment = "good"
  if y == 0:
    sentiment = "bad"

  return sentiment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Simulación de la llegada de nuevos datos

In [88]:
df_raw = pd.read_csv("/content/flipkart_review_data_2022_02.csv")
df_raw = df_raw.drop(columns="Unnamed: 0")

df_raw["sentiment"] = df_raw["reviewTitle"].apply(relabel_target)

In [89]:
df_test = pd.concat([df_raw[df_raw["sentiment"] == "bad"].sample(5),
                    df_raw[df_raw["sentiment"] == "good"].sample(5)])

In [90]:
df_test.groupby("sentiment")["sentiment"].count()

sentiment
bad     5
good    5
Name: sentiment, dtype: int64

In [91]:
import time

for index, row in df_test.iterrows():
  sentiment = main(pd.DataFrame(row).T)

  print(f"Label: {row['reviewTitle']}", f"-> Prediction: {sentiment}")

  time.sleep(1)

Label: Worst experience ever! -> Prediction: good
Label: Could be way better -> Prediction: bad
Label: Not recommended at all -> Prediction: bad
Label: Not recommended at all -> Prediction: good
Label: Very poor -> Prediction: bad
Label: Best in the market! -> Prediction: good
Label: Really Nice -> Prediction: good
Label: Really Nice -> Prediction: good
Label: Good quality product -> Prediction: good
Label: Terrific purchase -> Prediction: good


# Explanation of approach and decisions

El primer paso fue realizar fue realizar un filtrado de los comentarios, para obtener los comentarios de mejor calidad. Esto al eliminar el 1% de los comentarios con más dislikes.

El siguiente paso fue categorizar y crear una etiqueta para nuestros datos y puedan alimentar a los modelos, para ello utilicé como base, los titulos de los reviews.

Generé dos etiquetas: "good" y "bad". Al revisar la distribución de los datos, noté que la etiqueta "good" caracteriza a la mayoría de comentarios, por lo tanto los datos se encuentran desbalanceados.

El siguiente paso fue la obtención de los datos de train y test, normalmente se usa un 70% para los datos de entrenamiento y un 30% para los datos de test. Pero al contar con muy pocos registros, decidí usar 80-20, para tener una mayor cantidad de dastos para el entrenamiento.

Para tratar datos no balanceados, se puede:
- Realizar un sub-muestreo de la data que tiene mayor cantidad de datos
- Realizar un sobre-muestreo de la etiqueta con menor cantidad de datos
- Usar stratify cross validation
- Ensamblado de diferentes modelos, realizando un submuestreo de la categoría con mayor cantidad de datos y siempre usando la categoría con menos datos

Por el tiempo disponible opté por usar stratify cross validation que se integra dentro de la función GridSearch.

Tanto la columna tilte como reviewDescription fueron concatenadas, para ser utilizadas como "feature" para la construcción del modelo, el preprocesamiento incluyo:

- Eliminar signos de puntuación
- Convertir a minúsculas
- Remover palabras más comunes (stopwords)
- Eliminación de emojis
- Lematización: llevar a la raiz
- Remover palabras menores a 2 letras
- Vectorizar la información

Ya que es un problema de clasificación, use dos modelos de clasificación supervisada, Logistic Regression y XGB. Use diferentes hiperparametros que generaron diversos submodelos y poder evaluar una mayor cantidad de estos, mediante f1 score.

Una buen métrica para evaluar el modelo es f1-score, ya que los datos se encuentran desbalaceados

En ambos modelos tuvieron un buen desempeño con los datos de entrenamiento, pero no muy bien con los datos de train, por lo que se dio un overfitting (sobreentramiento).

El modelo Logistic Regression tiene un desempeño acetable.

En cuanto a oportunidades de mejora están:
- Utilizar los emojis dentro del modelo
- Tener una mayor cantidad de datos
- Evaluar la obtención de las etiquetas "good" y "bad"
- Probar técnias de reducción de dimensionalidad (PCA)
- Probar con otros parámetros al vectorizar con la función TfidfVectorizer
- Evaluar más algoritmos e hiperparametros: svm, knn, random forest, naives bayes
- Mejorar el score de manera general, y tener cuidado con el overfitting.
